In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

In [3]:
file_name = "NF-CICIDS2018-v3.parquet"
data = pd.read_parquet(file_name)

In [4]:
data.Label.value_counts()

Label
0    17514626
1     2600903
Name: count, dtype: int64

In [5]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [6]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [7]:
data.Attack.unique()

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DoS_attacks-GoldenEye', 'DoS_attacks-Slowloris',
       'DoS_attacks-SlowHTTPTest', 'DoS_attacks-Hulk',
       'DDoS_attacks-LOIC-HTTP', 'DDOS_attack-LOIC-UDP',
       'DDOS_attack-HOIC', 'Brute_Force_-Web', 'Brute_Force_-XSS',
       'SQL_Injection', 'Infilteration', 'Bot'], dtype=object)

In [8]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [9]:
data.groupby(by="Attack").count()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,1751463,1751463,1751463,1751463,1751463,1751463,1751463,1751463,1751463,1751463,...,1751463,1751463,1751463,1751463,1751463,1751463,1751463,1751463,1751463,1751463
Bot,20770,20770,20770,20770,20770,20770,20770,20770,20770,20770,...,20770,20770,20770,20770,20770,20770,20770,20770,20770,20770
Brute_Force_-Web,162,162,162,162,162,162,162,162,162,162,...,162,162,162,162,162,162,162,162,162,162
Brute_Force_-XSS,48,48,48,48,48,48,48,48,48,48,...,48,48,48,48,48,48,48,48,48,48
DDOS_attack-HOIC,103231,103231,103231,103231,103231,103231,103231,103231,103231,103231,...,103231,103231,103231,103231,103231,103231,103231,103231,103231,103231
DDOS_attack-LOIC-UDP,345,345,345,345,345,345,345,345,345,345,...,345,345,345,345,345,345,345,345,345,345
DDoS_attacks-LOIC-HTTP,28859,28859,28859,28859,28859,28859,28859,28859,28859,28859,...,28859,28859,28859,28859,28859,28859,28859,28859,28859,28859
DoS_attacks-GoldenEye,6130,6130,6130,6130,6130,6130,6130,6130,6130,6130,...,6130,6130,6130,6130,6130,6130,6130,6130,6130,6130
DoS_attacks-Hulk,10008,10008,10008,10008,10008,10008,10008,10008,10008,10008,...,10008,10008,10008,10008,10008,10008,10008,10008,10008,10008


In [10]:
# X = data.drop(columns=["Attack", "Label"])
X = data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [11]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [12]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [13]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [14]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
7702671,172.31.67.54,104.16.228.152,2.364280e-06,6.608158e-06,0.008995,1.171160e-04,0.011969,1.054044e-04,3.209025e-06,2.837709e-06,...,0.095941,0.341979,1.454227e-07,1.454227e-07,2.307945e-06,2.307945e-06,0.000000,1.514288e-06,"[2.3642799350579524e-06, 6.608157642632746e-06...",7702671
1769084,172.31.69.19,172.31.0.2,2.048333e-08,1.993613e-08,0.000076,1.413897e-06,0.000099,1.413897e-06,2.037054e-08,2.037054e-08,...,0.000000,0.000000,1.903185e-07,1.903185e-07,1.221542e-07,1.972363e-08,0.000085,1.828142e-07,"[2.0483332776817753e-08, 1.9936131278807104e-0...",1769084
633280,172.31.65.114,169.254.169.254,3.181417e-08,8.892055e-08,0.000059,7.879671e-07,0.000110,7.879671e-07,7.013197e-08,6.823303e-08,...,0.001291,0.002824,2.121297e-08,2.121297e-08,3.105612e-08,3.105612e-08,0.000000,2.037653e-08,"[3.1814172215512033e-08, 8.892054707761035e-08...",633280
11883761,172.31.65.65,172.31.0.2,7.122660e-09,6.932382e-09,0.000051,4.916537e-07,0.000114,4.916537e-07,7.083440e-09,7.083440e-09,...,0.000000,0.000000,6.617938e-08,6.617938e-08,5.099475e-08,6.858488e-09,0.000029,6.356990e-08,"[7.122660159886725e-09, 6.932382027330003e-09,...",11883761
19281635,172.31.64.95,172.31.0.2,1.111836e-08,1.082134e-08,0.000052,7.674635e-07,0.000114,7.674635e-07,1.105714e-08,1.105714e-08,...,0.000000,0.000000,1.033049e-07,1.033049e-07,7.054865e-08,1.070599e-08,0.000046,9.923159e-08,"[1.1118357717184076e-08, 1.0821336618881802e-0...",19281635


In [15]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [16]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [17]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [18]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [19]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [20]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [21]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [22]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

# dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [23]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [24]:
# Convert to GPU
# train_g = train_g.to('cuda')

In [25]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_CSE_v3.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [26]:
dgi.load_state_dict(torch.load('best_dgi_CSE_v3.pkl'))

<All keys matched successfully>

In [27]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [28]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [29]:
# Convert to GPU
# test_g = test_g.to('cuda')

In [30]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [31]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [32]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
from pyod.models.cblof import CBLOF
import gc

from tqdm import tqdm
import itertools

In [33]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [ ]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,-0.027911,-0.019412,0.016638,0.052916,0.105581,0.036157,-0.012026,-0.084199,0.031912,-0.083274,...,0.000000,0.000000,0.000000,0.000000,5.578918e-05,5.578918e-05,4.211242e-05,5.732556e-07,0.0,5.358939e-05
1,-0.027911,-0.019412,0.016638,0.052916,0.105581,0.036157,-0.012026,-0.084199,0.031912,-0.083274,...,0.000000,0.000000,0.000000,0.000000,5.602005e-05,5.602005e-05,4.037236e-05,5.756279e-07,0.0,5.381116e-05
2,-0.057264,-0.040439,-0.014924,0.046776,0.029506,0.011349,0.021992,0.013301,0.025913,-0.139302,...,0.000000,0.000000,0.771870,0.381833,6.274035e-06,6.274035e-06,9.185287e-06,9.185287e-06,0.0,6.026648e-06
3,-0.017759,-0.032197,0.034722,0.004706,0.047751,0.029255,0.000446,-0.074925,0.006789,-0.103674,...,0.000000,0.000014,0.039276,0.205106,6.453592e-07,6.453592e-07,9.448161e-07,9.448161e-07,0.0,6.199124e-07
4,-0.017759,-0.032197,0.034722,0.004706,0.047751,0.029255,0.000446,-0.074925,0.006789,-0.103674,...,0.000000,0.000014,0.039105,0.204210,6.425406e-07,6.425406e-07,9.406896e-07,9.406896e-07,0.0,6.172050e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203630,-0.029910,0.025817,-0.016431,0.002595,0.041463,0.062697,-0.023663,-0.107048,-0.023853,-0.107478,...,0.000000,0.000000,0.857455,0.423378,6.956678e-06,6.956678e-06,1.018469e-05,1.018469e-05,0.0,6.682373e-06
1203631,-0.031719,-0.003941,0.032492,0.039005,0.060259,0.031722,-0.022036,-0.062805,0.014916,-0.083344,...,0.000000,0.000000,0.004911,0.002475,2.623288e-06,2.623288e-06,3.840535e-06,3.840535e-06,0.0,2.519851e-06
1203632,-0.045501,0.034528,-0.037564,0.042303,0.056692,0.056418,-0.013252,-0.103189,-0.025277,-0.076668,...,0.000011,0.000000,0.708994,0.692388,1.456239e-06,1.456239e-06,2.131957e-06,2.131957e-06,0.0,1.398819e-06
1203633,-0.046385,0.007020,0.019926,0.065604,0.068039,0.028297,-0.036458,-0.048222,0.000296,-0.097017,...,0.000000,0.000000,0.045578,0.000000,4.202056e-07,4.202056e-07,6.151876e-07,6.151876e-07,0.0,4.036367e-07


In [ ]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
n_est = [5,6,7,9,10] # cant be lower than 5 or 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(benign_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [15:39<00:00, 31.32s/it]


{'n_estimators': 10, 'con': 0.1}
0.6692884534075014
              precision    recall  f1-score   support

           0     0.9180    0.8984    0.9081   1047581
           1     0.4034    0.4614    0.4305    156054

    accuracy                         0.8417   1203635
   macro avg     0.6607    0.6799    0.6693   1203635
weighted avg     0.8513    0.8417    0.8462   1203635



In [44]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(normal_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  3%|▎         | 1/30 [00:17<08:32, 17.66s/it]

5


100%|██████████| 30/30 [36:03<00:00, 72.10s/it]


{'n_estimators': 6, 'con': 0.2}
0.5418904801095815
              precision    recall  f1-score   support

           0     0.8866    0.8122    0.8477   1047581
           1     0.1935    0.3025    0.2360    156054

    accuracy                         0.7461   1203635
   macro avg     0.5400    0.5573    0.5419   1203635
weighted avg     0.7967    0.7461    0.7684   1203635



In [ ]:
# HBOS  Embeddings+Raw (Multimodal)

In [34]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [35]:
from pyod.models.hbos import HBOS

n_est = [50,55,60,65,70,75,80]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/42 [00:00<?, ?it/s]

100%|██████████| 42/42 [22:01<00:00, 31.47s/it]  


{'n_estimators': 80, 'con': 0.04}
0.4856515550367606
              precision    recall  f1-score   support

           0     0.8699    0.9549    0.9104   1047581
           1     0.1190    0.0409    0.0609    156054

    accuracy                         0.8364   1203635
   macro avg     0.4944    0.4979    0.4857   1203635
weighted avg     0.7725    0.8364    0.8002   1203635



In [36]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [17:20<00:00, 28.90s/it]


{'n_estimators': 10, 'con': 0.04}
0.48559048582568004
              precision    recall  f1-score   support

           0     0.8699    0.9558    0.9108   1047581
           1     0.1197    0.0404    0.0604    156054

    accuracy                         0.8371   1203635
   macro avg     0.4948    0.4981    0.4856   1203635
weighted avg     0.7726    0.8371    0.8005   1203635



In [37]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [38]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [10:53<00:00, 18.14s/it]


{'n_estimators': 10, 'con': 0.01}
0.4870324741968366
              precision    recall  f1-score   support

           0     0.8721    0.9895    0.9271   1047581
           1     0.2673    0.0258    0.0470    156054

    accuracy                         0.8645   1203635
   macro avg     0.5697    0.5076    0.4870   1203635
weighted avg     0.7937    0.8645    0.8130   1203635



In [39]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [12:07<00:00, 20.22s/it]


{'n_estimators': 10, 'con': 0.04}
0.4849912381267722
              precision    recall  f1-score   support

           0     0.8699    0.9570    0.9114   1047581
           1     0.1188    0.0389    0.0586    156054

    accuracy                         0.8380   1203635
   macro avg     0.4943    0.4980    0.4850   1203635
weighted avg     0.7725    0.8380    0.8008   1203635



In [40]:
##  IF  Emb+Raw

In [41]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [42]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:05<01:58,  5.16s/it]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:10<01:52,  5.09s/it]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:15<01:46,  5.08s/it]/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 17%

{'n_estimators': 100, 'con': 0.01}
0.491178569477881
              precision    recall  f1-score   support

           0     0.8726    0.9890    0.9271   1047581
           1     0.2918    0.0305    0.0552    156054

    accuracy                         0.8647   1203635
   macro avg     0.5822    0.5097    0.4912   1203635
weighted avg     0.7973    0.8647    0.8141   1203635



In [43]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [04:06<00:00, 10.28s/it]


{'n_estimators': 100, 'con': 0.04}
0.48625605933196225
              precision    recall  f1-score   support

           0     0.8701    0.9570    0.9115   1047581
           1     0.1232    0.0406    0.0611    156054

    accuracy                         0.8382   1203635
   macro avg     0.4966    0.4988    0.4863   1203635
weighted avg     0.7732    0.8382    0.8012   1203635

